In [ ]:
import napari
viewer = napari.Viewer()

from tqdm import tqdm
from tnia.deeplearning.dl_helper import quantile_normalization
import matplotlib.pyplot as plt
import numpy as np

import torch
from tnia.deeplearning.dl_helper import collect_training_data, divide_training_data
from tnia.plotting.plt_helper import imshow_multi2d
from skimage.io import imread

In [ ]:
cuda_present = torch.cuda.is_available()
ndevices = torch.cuda.device_count()
use_cuda = cuda_present and ndevices > 0
device = torch.device("cuda" if use_cuda else "cpu")  # "cuda:0" ... default device, "cuda:1" would be GPU index 1, "cuda:2" etc
print("number of devices:", ndevices, "\tchosen device:", device, "\tuse_cuda=", use_cuda)

In [ ]:
from pathlib import Path
import os

tnia_images_path = Path(r"D:\images")
parent_path = r'C:\Users\bnort\Documents\...'
parent_path = r'/home/bnorthan/besttestset/images/Semantic_Sparse/'

train_path = os.path.join(parent_path, 'patches')

image_patch_path = train_path + '/ground truth0'
label_patch_path = train_path + '/input0'

model_path = os.path.join(parent_path,'models')

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(image_patch_path):
    print('image_patch_path does not exist')

if not os.path.exists(label_patch_path):
    print('label_patch_path does not exist')

In [ ]:
dtype = torch.LongTensor

net = torch.load( Path(model_path) / 'model_Jan27_batchfix4.pth')
net = torch.load( Path(model_path) / 'ok.pth')
device = torch.device("cuda")
net=net.to(device)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = net.to(device)



In [ ]:
parent_path = Path(parent_path)
models_path = Path(parent_path) / 'models'

im_name = '20240905200636_SEP05ERLHS247450b.jpg'
im_name = '20240910210212_SEP10PRSP2424346.jpg'
#im_name = '20240911211757_SEP11EROSP245615.jpg'
#labels_name = '211122_AM_Al2O3_SE_021_sp_labels.tif'
im = imread(str(parent_path / (im_name)))

print(im.min(), im.max(), im.mean(), im.shape)
im = quantile_normalization(im)
print(im.min(), im.max(), im.mean(), im.shape)

im_to_test = im#[0:1024, 0:1024, :]



In [ ]:
im_to_test.dtype
net.eval()

In [ ]:
features = im_to_test.transpose(2,0,1).astype(np.float32)

features_tensor = torch.from_numpy(features).unsqueeze(0).to(device)       
#features = features.todevice()
with torch.no_grad():
    #predicted = net(features_tensor)
    outputs = []
    for chunk in torch.chunk(features_tensor, chunks=4, dim=3):  # Divide input into smaller parts
        
        with torch.no_grad():
            outputs.append(net(chunk))
        del chunk
        torch.cuda.empty_cache()
    predicted = torch.cat(outputs, dim=3)


print(predicted.shape, features.shape)

features = np.transpose(features, (1,2,0))

predicted.shape
c1 = predicted[0,0,:,:].cpu().detach().numpy()
c2 = predicted[0,1,:,:].cpu().detach().numpy()
c3 = predicted[0,2,:,:].cpu().detach().numpy()
fig = imshow_multi2d([features, c1, c2, c3], ['Image', 'Class 1', 'Class 2', 'Class 3'], 1, 4, 10, 10,colormaps=['gray', 'viridis', 'viridis', 'viridis'])

In [ ]:
import torch.nn.functional as F

# Apply softmax along the class dimension (dim=1)
probabilities = F.softmax(predicted, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)

p1 = probabilities[0,0,:,:].cpu().detach().numpy()
p2 = probabilities[0,1,:,:].cpu().detach().numpy()
p3 = probabilities[0,2,:,:].cpu().detach().numpy()
fig = imshow_multi2d([features, p1, p2, p3], ['Image', 'Class 1', 'Class 2', 'Class 3'], 1, 4, 10, 10,colormaps=['gray', 'viridis', 'viridis', 'viridis'])

print(probabilities.shape)

In [ ]:

edges = predicted_classes.cpu().detach().numpy()

edges = edges+1
edges[edges!=2] = 0
edges.shape
viewer.add_labels(edges, name='edges')

In [ ]:
viewer.add_image(features, contrast_limits=[0,1])
viewer.add_image(c1)
viewer.add_image(c2)
viewer.add_image(c3)
viewer.add_labels(predicted_classes[0].cpu().detach().numpy()+1)

In [ ]:
edges.shape

In [ ]:
viewer.add_labels(edges)